# Reprojection of image bboxes to a new projection with chosen resolution

If you don't have the required packages installed yet, run the stuff below first.

In [ ]:
# !conda install matplotlib cartopy
!pip install git+https://github.com/TAHMO/OpenRiverCamLib.git


Import the necessary packages

In [ ]:
%matplotlib inline
import OpenRiverCam as orc
import matplotlib.pyplot as plt
import cv2

# Extracting a frame from video and lens correcting it.
We have lens characteristics of the used camera stored in a variable called  `camera_type['lensParameters']` inside `example_data.py`. Lens distortions can be corrected with this. Let's extract one frame from a sample video (in `video_file`) and undistort it. We plot both distorted and undistorted. We use OpenCV to interact and manipualte video files and frames, and funcs of the OpenRiverCam lib (check docstrings to understand how they work. They are pretty well documented).

In [ ]:
video_file = "../examples/sample_limburg.mkv"
from example_data import camera_type
# open file for reading
cap = cv2.VideoCapture(video_file)
# extract the first frame (img will contain the frame)
success, img = cap.read()
img_corr = orc.cv._corr_lens(img, **camera_type["lensParameters"])

plt.figure(figsize=(18, 16))
plt.subplot(211)
plt.imshow(img)
plt.subplot(212)
plt.imshow(img_corr)


Now we can make a projection using some of our own control points, let's roughly use some controls over the staff gauge

In [ ]:
top_staff, bottom_staff, left_staff, right_staff = 0, 1.5, 0., 0.5
resolution = 0.005  # let us use a 1 cm resolution

src = [
    [580, 0],
    [580, 100],
    [610, 100],
    [610, 0],
]
dst = [
    [int(left_staff/resolution), int(top_staff/resolution)],
    [int(left_staff/resolution), int(bottom_staff/resolution)],
    [int(right_staff/resolution), int(bottom_staff/resolution)],
    [int(right_staff/resolution), int(top_staff/resolution)],
]


list(zip(*src))
plt.figure(figsize=(18, 10))
plt.imshow(img_corr)
plt.plot(*list(zip(*src)), "r")

# Make a transformation
open CV can transform the above image so that only the red box is returned, and it is warped so that the 4 coordinates are in fact the points as in `dst`. Thius requires a transformation function and a warp.

In [ ]:
# make transform object
M = orc.cv._get_M(src, dst)
cols = int((right_staff - left_staff) / resolution) # amount of columns. We compute tha
rows = int((bottom_staff - top_staff) / resolution)
img_warp = cv2.warpPerspective(img_corr, M, (cols, rows), flags=cv2.INTER_AREA)
img_warp


In [ ]:
plt.imshow(img_warp)

Yay a transformed image, with each pixel being 5 mm in length!